In [33]:
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
import requests
from matplotlib import font_manager
from mplsoccer import Pitch, VerticalPitch, FontManager, Sbopen
import seaborn as sns
import json
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import plotly.express as px
import os
import re
from tabulate import tabulate
from math import pi
from soccerplots.radar_chart import Radar
from mplsoccer import Radar, FontManager, grid



''' Set the folder path where the csv files are stored: '''
folder_path = r'C:\Users\Sander\Downloads\Wyscout_Prospect_Research-main1\Wyscout_Prospect_Research-main\Main App'

''' Get all the csv files in the folder: '''
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and ('23' in f or '24' in f or '25' in f)]


''' Create an empty list to store the dataframes: '''
dfs = []

''' Read all the csv files in the folder: '''
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)
    
''' Combine all the dataframes into one big dataframe: '''
combined_df = pd.concat(dfs,
                        ignore_index=True
                        )

''' Save the combined dataframe to a csv file: '''
# combined_df.to_csv(r'C:\Users\Sander\Downloads\combined_all_leagues.csv', index=False)




def get_player_id(combined_df):
    """
    Get the player ID for a specific player based on the name.
    
    Parameters:
    - player_name (str): The name of the player to search for.
    - combined_df (pd.DataFrame): The DataFrame containing player data.
    
    Returns:
    - str: The player ID if found, otherwise None.
    """
    # Filter DataFrame by player name
    
 
    user_input = input('Enter player name on the form x.x')
    player_of_interest = combined_df[combined_df['Player'] == user_input]
    
    # Check if any results are found
    if not player_of_interest.empty:
        # Print available player records
        print("Available player records:")
        print(player_of_interest[['Player', 'Wyscout id', 'Team']])
        
        # Ask the user to select the correct record if multiple are found
        if len(player_of_interest) > 1:
            try:
                choice = int(input(f"Enter the number of the correct record (1-{len(player_of_interest)}): ")) - 1
                if 0 <= choice < len(player_of_interest):
                    return player_of_interest.index[choice]
                else:
                    print("Invalid choice.")
                    return None
            except ValueError:
                print("Invalid input. Please enter a number.")
                return None
        else:
            # If only one record is found, return its ID
            return player_of_interest.index[0]
    else:
        print("Player not found.")
        return None

# Example usage
player_id = get_player_id(combined_df)
print(f"Selected player ID: {player_id}")




player_of_interest_2022_2023 = combined_df.iloc[player_id]

player_of_interest_2022_2023_t = pd.DataFrame(player_of_interest_2022_2023).T        

  



other_players_of_interest = combined_df[
    (combined_df['Position'].str.contains('cf', case=False)) &
    (combined_df['Primary position'].str.contains('cf', case=False)) &
    (combined_df['Minutes played'] >= 1500) &
    (combined_df['Age'] < 24) &
    (combined_df['Market value'] <= 5000000) &
    (combined_df['Successful attacking actions per 90'] >= player_of_interest_2022_2023_t['Successful attacking actions per 90'].values[0]*0.7) &
    # (central_stats_cm['Shots on target, %'] >= player_of_interest_central_stats['Shots on target, %'].values[0]*0.8) &
    # (central_stats_cm['Goal conversion, %'] >= player_of_interest_central_stats['Goal conversion, %'].values[0]*0.8) &
    (combined_df['Successful dribbles, %'] >= player_of_interest_2022_2023_t['Successful dribbles, %'].values[0]*0.7) &
    # (central_stats_cm['Accurate passes to penalty area, %'] >= player_of_interest_central_stats['Accurate passes to penalty area, %'].values[0]*0.8) &
    (combined_df['Accelerations per 90'] >= player_of_interest_2022_2023_t['Accelerations per 90'].values[0]*0.7) &
    (combined_df['Offensive duels won, %'] >= player_of_interest_2022_2023_t['Offensive duels won, %'].values[0]*0.7) &
    (combined_df['Progressive runs per 90'] >= player_of_interest_2022_2023_t['Progressive runs per 90'].values[0]*0.7) &
    # (central_stats_cm['Goals'] >= player_of_interest_central_stats['Goals'].values[0]*0.5) &
    (combined_df['Deep completions per 90'] >= player_of_interest_2022_2023_t['Deep completions per 90'].values[0]*0.7) 
    # (combined_df['Touches in box per 90'] >= player_of_interest_2022_2023_t['Touches in box per 90'].values[0]*0.8)
].drop_duplicates()


position_input = input('Select position from: CF, W, AM, CM, DM, WB, CB, GK')
if position_input == 'CF':
    params = ['Goals per 90', 'xG', 'Non-penalty goals per 90', 'Shots on target, %', 'Goal conversion, %', 'Successful attacking actions per 90', 'Progressive runs per 90', 'Touches in box per 90']
elif position_input == 'W' or position_input == 'AM':
    params = ['Successful attacking actions per 90','Successful dribbles, %', 'Accelerations per 90', 
'Offensive duels won, %', 'Progressive runs per 90', 'Touches in box per 90']

ranges = []
min_val = []
max_val = []
for x in params:
    min_val.append(other_players_of_interest[x].min())
    max_val.append(other_players_of_interest[x].max())
    # ranges.append((min_val - (0.25 * min_val), max_val + (0.25 * max_val)))
    
radar = Radar(params, min_val, max_val,
            # whether to round any of the labels to integers instead of decimal places
            round_int=[False]*len(params),
            num_rings=4,  # the number of concentric circles (excluding center circle)
            # if the ring_width is more than the center_circle_radius then
            # the center circle radius will be wider than the width of the concentric circles
            ring_width=1, center_circle_radius=1)        
        
URL1 = ('https://raw.githubusercontent.com/googlefonts/SourceSerifProGFVersion/main/fonts/'
        'SourceSerifPro-Regular.ttf')
serif_regular = FontManager(URL1)
URL2 = ('https://raw.githubusercontent.com/googlefonts/SourceSerifProGFVersion/main/fonts/'
        'SourceSerifPro-ExtraLight.ttf')
serif_extra_light = FontManager(URL2)
URL3 = ('https://raw.githubusercontent.com/google/fonts/main/ofl/rubikmonoone/'
        'RubikMonoOne-Regular.ttf')
rubik_regular = FontManager(URL3)
URL4 = 'https://raw.githubusercontent.com/googlefonts/roboto/main/src/hinted/Roboto-Thin.ttf'
robotto_thin = FontManager(URL4)
URL5 = ('https://raw.githubusercontent.com/google/fonts/main/apache/robotoslab/'
        'RobotoSlab%5Bwght%5D.ttf')
robotto_bold = FontManager(URL5)


def save_figure(fig, folder_path, filename):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
          # Create the full file path
    file_path = os.path.join(folder_path, filename)

    # Save the figure
    fig.savefig(file_path)
    plt.close(fig)  # Close the figure after saving
    

def plot_radar():
    for i in range(len(player_of_interest_2022_2023_t)):
        
        player = player_of_interest_2022_2023_t[params].values[0]

        # creating the figure using the grid function from mplsoccer:
        fig, axs = grid(figheight=14, grid_height=0.915, title_height=0.06, endnote_height=0.025,
                        title_space=0, endnote_space=0, grid_key='radar', axis=False)

        # plot the radar
        radar.setup_axis(ax=axs['radar'], facecolor='None')
        rings_inner = radar.draw_circles(ax=axs['radar'], facecolor='#28252c', edgecolor='#39353f', lw=1.5)
        radar_output = radar.draw_radar(player, ax=axs['radar'],
                                                kwargs_radar={'facecolor': '#FFD700', 'alpha': 0.6},
                                                kwargs_rings={'facecolor': '#FF1493', 'alpha': 0})
        radar_poly, rings_outer, vertices1 = radar_output
        range_labels = radar.draw_range_labels(ax=axs['radar'], fontsize=25, color='#fcfcfc',
                                            fontproperties=robotto_thin.prop)
        param_labels = radar.draw_param_labels(ax=axs['radar'], fontsize=25, color='#fcfcfc',
                                            fontproperties=robotto_thin.prop)
        axs['radar'].scatter(vertices1[:, 0], vertices1[:, 1],
                            c='#FFD700', edgecolors='#6d6c6d', marker='o', s=150, zorder=2)

        # adding the endnote and title text (these axes range from 0-1, i.e. 0, 0 is the bottom left)
        # Note we are slightly offsetting the text from the edges by 0.01 (1%, e.g. 0.99)
        endnote_text = axs['endnote'].text(0.99, 0.5, '@Fotbadlstat on X',
                                        color='#a3c2a8', fontproperties=rubik_regular.prop,
                                        fontsize=13, ha='right', va='center')
        title1_text = axs['title'].text(0.01, 0.65, player_of_interest_2022_2023_t['Player'].values[0], fontsize=25,
                                        fontproperties=robotto_bold.prop,
                                        ha='left', va='center', color='#FFD700')
        title2_text = axs['title'].text(0.01, 0.25, f'{player_of_interest_2022_2023_t['Team within selected timeframe'].values[0]} | {player_of_interest_2022_2023_t['Age'].values[0]} years old', fontsize=20,
                                        fontproperties=robotto_thin.prop,
                                        ha='left', va='center', color='#FFD700')
        title3_text = axs['title'].text(0.99, 0.65, f'Primary position: {player_of_interest_2022_2023_t['Primary position'].values[0]}', fontsize=25,
                                        fontproperties=robotto_bold.prop,
                                        ha='right', va='center', color='#FFD700')
        title4_text = axs['title'].text(0.99, 0.25, f'Minutes played: {player_of_interest_2022_2023_t['Minutes played'].values[0]}', fontsize=20,
                                        fontproperties=robotto_thin.prop,
                                        ha='right', va='center', color='#FFD700')

        fig.set_facecolor('#121212')
        
        
        # Create the folder path
        
        player_folder = f"{player_of_interest_2022_2023_t['Player'].values[0]}"
        folder_path = os.path.join(r'C:\Users\Sander\Documents\football_analytics\scouting', player_folder)

        # Create the filename and save the figure
        filename = f"{player_of_interest_2022_2023_t['Player'].values[0]}_{player_id}.png"
        
        # Save the figure and ensure folder exists
        save_figure(fig, folder_path, filename)
        


plot_radar()

Available player records:
           Player  Wyscout id        Team
84530   O. Hintsa      -57441  Falkenberg
148221  O. Hintsa      -57441     Sogndal
Selected player ID: 148221
